In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from textPreProcessor import *
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math
import dateparser

2023-06-15 10:24:55.535574: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


importing Jupyter notebook from textPreProcessor.ipynb
Original Date ----> After Processing 

5-12-22
True
2-2-2023
True
5-12-2023
True
5-05-2023
True
5-05-2023
True
10-24-2022
True
10-24-2022
True
22-05-2023
True
21-02-23
True
5-12-22
True
31-08-2022
True
importing Jupyter notebook from imagePreProcessor.ipynb
importing Jupyter notebook from TesseractModify.ipynb


NameError: name 'pwd' is not defined

In [ ]:
imagesFolderPath = '/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv4/Dataset/SampleImages/'

In [2]:
def preProcessDataFrame(df, image):
    #drop null values
    df.dropna(inplace=True)
    
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"] 
    df["y"] = df["top"] + df["height"]
    
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]
    
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0

In [3]:
def findDate(dateDF):
    dates = []
    
    format = "%d-%m-%Y"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
            
    format = "%m-%d-%Y"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
            
    format = "%y-%m-%d"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
            
    return dates

In [4]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = preProcessText(text)
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [5]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [6]:
def allocateNeighbours(df, neighbourFields):
    for index in df.index:
        x1 = df["x"][index]/df["PageWidth"][index]
        y1 = df["y"][index]/df["PageHeight"][index]
        block = df["block_num"][index]
        neighbours = {}
        for index2 in df.index:
            x2 = df["x"][index2]/df["PageWidth"][index2]
            y2 = df["y"][index2]/df["PageHeight"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if dist<0.2:
                neighbours[df["text"][index2].lower()]=dist
            if abs(y1-y2)<0.2 and abs(block2-block)<=1<=1:
                neighbours[df["text"][index2].lower()] = dist
        for n in neighbourFields:
            if n in neighbours:
                df[n][index]=neighbours[n]

In [7]:
def addNeighbours(df):
    neighbourFields = ["date", "dated", "invoice", "delivery", "order", "due", "payment", "tax", "bill", "receipt", "issue"]
    for col in neighbourFields:
        df[col]=0
    allocateNeighbours(df, neighbourFields)
    

In [8]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [9]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [10]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [11]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [12]:
dfList = batchCreate(imagesFolderPath)

NameError: name 'imagesFolderPath' is not defined

In [ ]:
len(dfList)

In [ ]:
resultDF = pd.concat(dfList)

In [ ]:
resultDF.shape

In [ ]:
len(resultDF["imageName"].unique())

In [ ]:
resultDF

In [ ]:
resultDF.reset_index(inplace=True)

In [ ]:
resultDF.drop(columns=["index"], inplace=True)

In [ ]:
resultDF.head()

In [ ]:
pwd

In [ ]:
os.chdir('/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv4/Dataset/')

In [ ]:
pwd

In [ ]:
resultDF.to_csv("tesseract.csv")